In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_columns=None

In [172]:
DOUBLE11_DAY = 184
VALID_SET_SIZE = 0.2
PCA_COMPONENTS = 5
RANDOM_SEED = 42
N_FEATURES = 150
EPSILON = 1e-10
SPLITS = 10

In [173]:
duff=pd.read_csv('processed_data.csv')

In [174]:
duff[duff['time_stamp']==1111]

,user_id,age_range,gender,merchant_id,label,item_id,category_id,brand_id,time_stamp,action_type
27,221313,3,1,1110,-1,857,1112,2187,1111,0
44,226947,5,1,2116,-1,867,656,7334,1111,0
45,35460,3,0,1393,-1,862,1389,4705,1111,0
63,358791,3,0,2217,-1,919,1467,3012,1111,0
75,64905,3,0,1393,-1,862,1389,4705,1111,0
...,...,...,...,...,...,...,...,...,...,...
4606,381302,4,0,3058,-1,809,308,8473,1111,0
4609,305528,0,0,4850,-1,861,812,7863,1111,0
4622,27515,3,1,4644,-1,876,267,8235,1111,0
4623,228731,6,0,1110,-1,857,1112,2187,1111,0


In [175]:
df=pd.read_csv('df_t.csv')
df_user_info=pd.read_csv('df_info.csv')
df_user_log=pd.read_csv('df_log.csv')

In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4644 entries, 0 to 4643
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      4644 non-null   int64
 1   merchant_id  4644 non-null   int64
 2   label        4644 non-null   int64
dtypes: int64(3)
memory usage: 109.0 KB


In [177]:
df_user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4644 entries, 0 to 4643
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id    4644 non-null   int64
 1   age_range  4644 non-null   int64
 2   gender     4644 non-null   int64
dtypes: int64(3)
memory usage: 109.0 KB


In [178]:
df_user_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4644 entries, 0 to 4643
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      4644 non-null   int64
 1   item_id      4644 non-null   int64
 2   category_id  4644 non-null   int64
 3   merchant_id  4644 non-null   int64
 4   brand_id     4644 non-null   int64
 5   time_stamp   4644 non-null   int64
 6   action_type  4644 non-null   int64
dtypes: int64(7)
memory usage: 254.1 KB


In [179]:
df_user_log.head()

,user_id,item_id,category_id,merchant_id,brand_id,time_stamp,action_type
0,34944,867,656,2116,7334,1017,0
1,252288,825,662,3990,5644,819,0
2,210048,866,1213,4255,1573,711,2
3,210048,866,1213,4255,1573,711,0
4,210048,866,1213,4255,1573,711,0


In [180]:
df_user_log.rename(columns={'seller_id' : 'merchant_id'}, inplace=True)
df_user_log.rename(columns={'category_id' : 'cat_id'}, inplace=True)

In [181]:
df_user_log['time_stamp'] = (pd.to_datetime(df_user_log['time_stamp'], format='%m%d') - pd.to_datetime(df_user_log['time_stamp'].min(), format='%m%d')).dt.days

In [182]:
df_user_log.head()

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
0,34944,867,656,2116,7334,159,0
1,252288,825,662,3990,5644,100,0
2,210048,866,1213,4255,1573,61,2
3,210048,866,1213,4255,1573,61,0
4,210048,866,1213,4255,1573,61,0


In [183]:
df_user_log['time_stamp'] = df_user_log['time_stamp'].astype(np.int16)

In [184]:
users = df_user_log.groupby('user_id')

In [185]:
merchants = df_user_log.groupby('merchant_id')

In [186]:
users_merchants = df_user_log.groupby(['user_id', 'merchant_id'])

In [187]:
df_user_log['time_period'] = df_user_log['time_stamp'] // 31

In [188]:
len(df_user_log)

4644

In [189]:
len(df)

4644

In [190]:
df.head()

,user_id,merchant_id,label
0,34944,2116,-1
1,252288,3990,-1
2,210048,4255,-1
3,210048,4255,-1
4,210048,4255,-1


In [191]:
df_user_info.head()

,user_id,age_range,gender
0,34944,5,0
1,252288,3,0
2,210048,3,1
3,210048,3,1
4,210048,3,1


In [192]:
len(df_user_info)

4644

In [193]:
df_user_info.drop_duplicates(inplace=True)

In [194]:
len(df_user_info)

2447

In [195]:
# transform age categorical features into different binary features
to_merge = pd.get_dummies(df_user_info, prefix='age', columns=['age_range'])

In [196]:
to_merge.head()

,user_id,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8
0,34944,0,0,0,0,0,1,0,0,0
1,252288,0,0,0,1,0,0,0,0,0
2,210048,1,0,0,1,0,0,0,0,0
6,279168,1,0,1,0,0,0,0,0,0
21,217728,0,0,0,0,1,0,0,0,0


In [197]:
len(to_merge)

2447

In [198]:
df= df.merge(to_merge, on='user_id', how='left')

In [199]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8
0,34944,2116,-1,0,0,0,0,0,1,0,0,0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0


In [200]:
len(df)

4644

In [201]:
to_merge = users.nunique().reset_index().rename(columns={
    'item_id': 'items_user', 
    'cat_id': 'categories_user',
    'merchant_id': 'merchants_user',
    'brand_id': 'brands_user',
    'time_stamp': 'dates_user',
    'time_period': 'periods_user',
    'action_type': 'action_types_user'
    })

In [202]:
df = df.merge(to_merge, on='user_id', how='left')

In [203]:
to_merge = merchants.nunique().reset_index().rename(columns={
    'item_id': 'items_merchant', 
    'cat_id': 'categories_merchant',
    'user_id': 'users_merchant',
    'brand_id': 'brands_merchant',
    'time_stamp': 'dates_merchant',
    'time_period': 'periods_merchant',
    'action_type': 'action_types_merchant'
    })

In [204]:
to_merge.head()

,merchant_id,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant
0,67,1,1,1,1,1,1,1
1,94,1,1,1,1,1,1,1
2,111,5,1,1,1,5,1,4
3,145,3,1,1,1,3,1,2
4,173,28,1,1,1,25,2,4


In [209]:
to_merge[to_merge['items_merchant']==2]

,merchant_id,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant
24,778,2,2,2,1,2,1,1
43,1429,3,2,1,1,3,1,2
67,2476,3,2,1,1,1,1,1
109,4363,3,2,1,2,3,1,2
125,4818,98,2,2,1,77,3,6


In [211]:
df_user_log[df_user_log['merchant_id']==4363]

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type,time_period
1895,227008,844,895,4363,6831,180,0,5
3011,403844,844,895,4363,6831,119,0,3
4140,7220,858,895,4363,7651,184,0,5
4141,7220,858,895,4363,7651,184,0,5


In [205]:
df = df.merge(to_merge, on='merchant_id', how='left')

In [212]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5


In [218]:
to_merge = users_merchants.nunique().reset_index().rename(columns={
    'item_id': 'items_user_merchant', 
    'cat_id': 'categories_user_merchant',
    'brand_id': 'brands_user_merchant',
    'time_stamp': 'dates_user_merchant',
    'time_period': 'periods_user_merchant',
    'action_type': 'action_types_user_merchant'
    })

In [227]:
to_merge.head()

,user_id,merchant_id,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant
0,170,4255,1,1,1,2,1,2
1,393,4344,1,1,1,1,1,1
2,699,3491,1,1,1,1,1,1
3,784,983,1,1,1,1,1,1
4,867,4255,1,1,1,1,1,1


In [239]:
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')

In [241]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639,230015,4255,-1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1
4640,230015,4255,-1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1
4641,64895,3491,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,22,1,1,1,11,1,1,1,1,1,1,1,1
4642,5759,4255,-1,0,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1,681,1,1,1,103,3,5,1,1,1,1,1,1


In [242]:
to_merge = users['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_user',
    1: 'carts_user',
    2: 'purchases_user',
    3: 'favourites_user'
    })

In [248]:
df = df.merge(to_merge, on='user_id', how='left')

In [250]:
to_merge = merchants['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_merchant', 
    1: 'carts_merchant',
    2: 'purchases_merchant',
    3: 'favourites_merchant'
    })

In [256]:
df = df.merge(to_merge, on='merchant_id', how='left')

In [259]:
to_merge = users_merchants['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_user_merchant',
    1: 'carts_user_merchant',
    2: 'purchases_user_merchant',
    3: 'favourites_user_merchant'
    })

In [266]:
df= df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')

In [269]:
df['clicks_in_merchant_ratio_perspective'] = df['clicks_user_merchant'] / (df['clicks_user'] + EPSILON)

In [273]:
df['purchases_in_merchant_ratio_perspective'] = df['purchases_user_merchant'] / (df['purchases_user'] + EPSILON)

In [274]:
df['favourites_in_merchant_ratio_perspective'] = df['favourites_user_merchant'] / (df['favourites_user'] + EPSILON)

In [276]:
df['clicks_by_user_ratio_perspective'] = df['clicks_user_merchant'] / (df['clicks_merchant'] + EPSILON)

In [277]:
df['purchases_by_user_ratio_perspective'] = df['purchases_user_merchant'] / (df['purchases_merchant'] + EPSILON)

In [278]:
df['favourites_by_user_ratio_perspective'] = df['favourites_user_merchant'] / (df['favourites_merchant'] + EPSILON)

In [279]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0


In [282]:
df['clicks_user_ratio'] = df['clicks_user'] / (df['clicks_user'] + df['purchases_user'] + df['favourites_user'] + EPSILON)

In [283]:
df['purchases_user_ratio'] = df['purchases_user'] / (df['clicks_user'] + df['purchases_user'] + df['favourites_user'] + EPSILON)

In [284]:
df['favourites_user_ratio'] = df['favourites_user'] / (df['clicks_user'] + df['purchases_user'] + df['favourites_user'] + EPSILON)

In [285]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.00,0.00,0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.00,0.00,0.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0


In [287]:
df['clicks_user_merchant_ratio'] = df['clicks_user_merchant'] / (df['clicks_user_merchant']+ df['purchases_user_merchant'] + df['favourites_user_merchant'] + EPSILON)

In [288]:
df['purchases_user_merchant_ratio'] = df['purchases_user_merchant'] / (df['clicks_user_merchant'] + df['purchases_user_merchant'] + df['favourites_user_merchant'] + EPSILON)

In [289]:
df['favourites_user_merchant_ratio'] = df['favourites_user_merchant'] / (df['clicks_user_merchant'] +  df['purchases_user_merchant'] + df['favourites_user_merchant'] + EPSILON)

In [291]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0


In [292]:
df['clicks_merchant_ratio'] = df['clicks_merchant'] / (df['clicks_merchant'] + df['purchases_merchant'] + df['favourites_merchant'] + EPSILON)

In [294]:
df['purchases_merchant_ratio'] = df['purchases_merchant'] / (df['clicks_merchant'] + df['purchases_merchant'] + df['favourites_merchant'] + EPSILON)

In [295]:
df['favorites_merchant_ratio'] = df['favourites_merchant'] / (df['clicks_merchant'] + df['purchases_merchant'] + df['favourites_merchant'] + EPSILON)

In [297]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.929412,0.041176,0.029412
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.942222,0.013333,0.044444
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831


In [302]:
to_merge = (users['time_stamp'].max() - users['time_stamp'].min()).rename('interval')

In [310]:
df= df.merge(to_merge, on='user_id', how='left')

# Monthly Features

###### users monthly

In [314]:
users_time = df_user_log.groupby(['user_id', 'time_period'])

In [338]:
to_merge = users_time['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_user',
    1: 'carts_user',
    2: 'purchases_user',
    3: 'favourites_user'
    }).reset_index()

In [343]:
to_merge

action_type,user_id,time_period,clicks_user,purchases_user,favourites_user
0,170,2,1,0,0
1,170,3,1,0,0
2,393,5,1,0,0
3,699,0,1,0,0
4,784,2,1,0,0
...,...,...,...,...,...
2609,422954,4,1,0,0
2610,422967,0,2,0,0
2611,423225,5,1,0,0
2612,423387,1,1,0,0


In [347]:
to_merge.groupby('user_id').max()

action_type,time_period,clicks_user,purchases_user,favourites_user
user_id,,,,
170,3,1,0,0
393,5,1,0,0
699,0,1,0,0
784,2,1,0,0
867,5,1,0,0
...,...,...,...,...
422954,4,1,0,0
422967,0,2,0,0
423225,5,1,0,0


In [344]:
to_merge_aux = to_merge.groupby('user_id').max().drop('time_period', axis=1).rename(columns={
    'clicks_user': 'clicks_user_period_max',
    'purchases_user': 'purchases_user_period_max',
    'favourites_user': 'favourites_user_period_max'
})

In [353]:
df= df.merge(to_merge_aux, on='user_id', how='left')

In [359]:
len(df)

4644

In [363]:
to_merge.head()

action_type,user_id,time_period,clicks_user,purchases_user,favourites_user
0,170,2,1,0,0
1,170,3,1,0,0
2,393,5,1,0,0
3,699,0,1,0,0
4,784,2,1,0,0


In [365]:
to_merge_aux = to_merge.groupby('user_id').mean().drop('time_period', axis=1).rename(columns={
    'clicks_user': 'clicks_user_period_mean',
    'purchases_user': 'purchases_user_period_mean',
    'favourites_user': 'favourites_user_period_mean'
})

In [368]:
df= df.merge(to_merge_aux, on='user_id', how='left')

In [369]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0


In [370]:
to_merge_aux = to_merge.groupby('user_id').std().drop('time_period', axis=1).rename(columns={
    'clicks_user': 'clicks_user_period_std',
    'carts_user': 'carts_user_period_std',
    'purchases_user': 'purchases_user_period_std',
    'favourites_user': 'favourites_user_period_std'
}).fillna(0)

In [376]:
df = df.merge(to_merge_aux, on='user_id', how='left')

In [377]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0


In [379]:
to_merge_aux = to_merge.groupby('user_id').median().drop('time_period', axis=1).rename(columns={
    'clicks_user': 'clicks_user_period_median',
    'carts_user': 'carts_user_period_median',
    'purchases_user': 'purchases_user_period_median',
    'favourites_user': 'favourites_user_period_median'
})

In [380]:
df = df.merge(to_merge_aux, on='user_id', how='left')

In [382]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0


In [384]:
to_merge.head()

action_type,user_id,time_period,clicks_user,purchases_user,favourites_user
0,170,2,1,0,0
1,170,3,1,0,0
2,393,5,1,0,0
3,699,0,1,0,0
4,784,2,1,0,0


In [394]:
to_merge = to_merge.groupby(['user_id', 'time_period']).sum().unstack().fillna(0).stack().reset_index()

In [399]:
to_merge = to_merge.pivot_table(values=['clicks_user', 'purchases_user', 'favourites_user'], index='user_id', columns='time_period')

In [401]:
to_merge.head()

action_type clicks_user                          favourites_user            \
time_period           0    1    2    3    4    5               0    1    2   
user_id                                                                      
170                 0.0  0.0  1.0  1.0  0.0  0.0             0.0  0.0  0.0   
393                 0.0  0.0  0.0  0.0  0.0  1.0             0.0  0.0  0.0   
699                 1.0  0.0  0.0  0.0  0.0  0.0             0.0  0.0  0.0   
784                 0.0  0.0  1.0  0.0  0.0  0.0             0.0  0.0  0.0   
867                 0.0  0.0  0.0  0.0  0.0  1.0             0.0  0.0  0.0   

action_type                purchases_user                           
time_period    3    4    5              0    1    2    3    4    5  
user_id                                                             
170          0.0  0.0  0.0            0.0  0.0  0.0  0.0  0.0  0.0  
393          0.0  0.0  0.0            0.0  0.0  0.0  0.0  0.0  0.0  
699          0.0  0.0  0.0            0.0  0.0  0.0  0.0  0.0  0.0  
784          0.0  0.0  0.0            0.0  0.0  0.0  0.0  0.0  0.0  
867          0.0  0.0  0.0            0.0  0.0  0.0  0.0  0.0  0.0

MultiIndex([(    'clicks_user', 0),
            (    'clicks_user', 1),
            (    'clicks_user', 2),
            (    'clicks_user', 3),
            (    'clicks_user', 4),
            (    'clicks_user', 5),
            ('favourites_user', 0),
            ('favourites_user', 1),
            ('favourites_user', 2),
            ('favourites_user', 3),
            ('favourites_user', 4),
            ('favourites_user', 5),
            ( 'purchases_user', 0),
            ( 'purchases_user', 1),
            ( 'purchases_user', 2),
            ( 'purchases_user', 3),
            ( 'purchases_user', 4),
            ( 'purchases_user', 5)],
           names=['action_type', 'time_period'])

In [403]:
to_merge.columns.values

array([('clicks_user', 0), ('clicks_user', 1), ('clicks_user', 2),
       ('clicks_user', 3), ('clicks_user', 4), ('clicks_user', 5),
       ('favourites_user', 0), ('favourites_user', 1),
       ('favourites_user', 2), ('favourites_user', 3),
       ('favourites_user', 4), ('favourites_user', 5),
       ('purchases_user', 0), ('purchases_user', 1),
       ('purchases_user', 2), ('purchases_user', 3),
       ('purchases_user', 4), ('purchases_user', 5)], dtype=object)

In [405]:
co=('clicks_user', 0)

In [406]:
for x in co:
    print(x)

clicks_user
0


In [408]:
to_merge.columns = ['_period_'.join(str(x) for x in col) for col in to_merge.columns.values]

In [410]:
to_merge.head()

,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5
user_id,,,,,,,,,,,,,,,,,,
170,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
393,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
699,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
784,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
867,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [411]:
len(to_merge)

2447

In [412]:
df = df.merge(to_merge, on='user_id', how='left')

In [417]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


###### users trendline

In [418]:
def trendline(x, y):
    n = x.shape[1]
    sum_x = np.sum(x)
    sum_x_2 = np.sum(x**2)
    sum_y = np.sum(y, axis=1)
    u = (n * np.dot(y, x.T)) - (sum_x * sum_y).reshape(-1, 1)
    l = np.sum(sum_x_2) - (sum_x ** 2)
    return (u / l)

In [419]:
purchase_col = [col for col in df if col.startswith('purchases_user_period') and col[-1].isdigit()]

In [420]:
purchase_col

['purchases_user_period_0',
 'purchases_user_period_1',
 'purchases_user_period_2',
 'purchases_user_period_3',
 'purchases_user_period_4',
 'purchases_user_period_5']

In [424]:
x = np.array([int(col[-1]) for col in purchase_col]).reshape(1, -1)

In [425]:
x

array([[0, 1, 2, 3, 4, 5]])

In [426]:
y = df[purchase_col].to_numpy()

In [427]:
y

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [428]:
df['purchase_user_trendline'] = trendline(x, y)

In [438]:
clicks_col = [col for col in df if col.startswith('clicks_user_period') and col[-1].isdigit()]

In [439]:
y = df[clicks_col].to_numpy()

In [440]:
df['clicks_user_trendline'] = trendline(x, y)

In [444]:
favourites_col = [col for col in df if col.startswith('favourites_user_period') and col[-1].isdigit()]

In [445]:
y = df[favourites_col].to_numpy()

In [446]:
df['favourites_user_trendline'] = trendline(x, y)

In [447]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.017647,-0.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639,230015,4255,-1,0,0,0,0,1,0,0,0,0,1,1,1,

###### Merchant monthly

In [448]:
merchants_time = df_user_log.groupby(['merchant_id', 'time_period'])

In [450]:
to_merge = merchants_time['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_merchant',
    1: 'carts_merchant',
    2: 'purchases_merchant',
    3: 'favourites_merchant'
    }).reset_index()

In [455]:
to_merge

action_type,merchant_id,time_period,clicks_merchant,purchases_merchant,favourites_merchant
0,67,0,1,0,0
1,94,3,1,0,0
2,111,0,1,0,0
3,111,1,1,0,0
4,111,2,1,0,0
...,...,...,...,...,...
275,4945,1,8,0,1
276,4945,2,1,0,1
277,4945,3,2,0,0
278,4945,4,2,0,0


In [453]:
to_merge_aux = to_merge.groupby('merchant_id').max().drop('time_period', axis=1).rename(columns={
    'clicks_merchant': 'clicks_merchant_period_max',
    'carts_merchant': 'carts_merchant_period_max',
    'purchases_merchant': 'purchases_merchant_period_max',
    'favourites_merchant': 'favourites_merchant_period_max'
})

In [456]:
df = df.merge(to_merge_aux, on='merchant_id', how='left')

In [458]:
df.head()

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline,clicks_merchant_period_max,purchases_merchant_period_max,favourites_merchant_period_max
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0,154,7,5
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.00,0.00,0.0,1.00,0.00,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.017647,-0.0,80,2,6
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14


In [460]:
to_merge_aux = to_merge.groupby('merchant_id').mean().drop('time_period', axis=1).rename(columns={
    'clicks_merchant': 'clicks_merchant_period_mean',
    'carts_merchant': 'carts_merchant_period_mean',
    'purchases_merchant': 'purchases_merchant_period_mean',
    'favourites_merchant': 'favourites_merchant_period_mean'
})

In [461]:
df = df.merge(to_merge_aux, on='merchant_id', how='left')

In [462]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline,clicks_merchant_period_max,purchases_merchant_period_max,favourites_merchant_period_max,clicks_merchant_period_mean,purchases_merchant_period_mean,favourites_merchant_period_mean
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0,154,7,5,79.0,3.5,2.5
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.017647,-0.0,80,2,6,42.4,0.6,2.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14,291.4,92.8,4.6
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14,291.4,92.8,4.6
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14,291.4,92.8,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [463]:
to_merge_aux = to_merge.groupby('merchant_id').std().drop('time_period', axis=1).rename(columns={
    'clicks_merchant': 'clicks_merchant_period_std',
    'carts_merchant': 'carts_merchant_period_std',
    'purchases_merchant': 'purchases_merchant_period_std',
    'favourites_merchant': 'favourites_merchant_period_std'
}).fillna(0)

In [464]:
df = df.merge(to_merge_aux, on='merchant_id', how='left')

In [465]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline,clicks_merchant_period_max,purchases_merchant_period_max,favourites_merchant_period_max,clicks_merchant_period_mean,purchases_merchant_period_mean,favourites_merchant_period_mean,clicks_merchant_period_std,purchases_merchant_period_std,favourites_merchant_period_std
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0,154,7,5,79.0,3.5,2.5,106.066017,4.949747,3.535534
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.017647,-0.0,80,2,6,42.4,0.6,2.0,27.272697,0.894427,2.449490
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14,291.4,92.8,4.6,224.550217,78.023714,5.458938
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14,291.4,92.8,4.6,224.550217,78.023714,5.458938
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.

In [466]:
to_merge_aux = to_merge.groupby('merchant_id').median().drop('time_period', axis=1).rename(columns={
    'clicks_merchant': 'clicks_merchant_period_median',
    'carts_merchant': 'carts_merchant_period_median',
    'purchases_merchant': 'purchases_merchant_period_median',
    'favourites_merchant': 'favourites_merchant_period_median'
})

In [467]:
df = df.merge(to_merge_aux, on='merchant_id', how='left')

In [468]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline,clicks_merchant_period_max,purchases_merchant_period_max,favourites_merchant_period_max,clicks_merchant_period_mean,purchases_merchant_period_mean,favourites_merchant_period_mean,clicks_merchant_period_std,purchases_merchant_period_std,favourites_merchant_period_std,clicks_merchant_period_median,purchases_merchant_period_median,favourites_merchant_period_median
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0,154,7,5,79.0,3.5,2.5,106.066017,4.949747,3.535534,79.0,3.5,2.5
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.017647,-0.0,80,2,6,42.4,0.6,2.0,27.272697,0.894427,2.449490,43.0,0.0,2.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14,291.4,92.8,4.6,224.550217,78.023714,5.458938,395.0,96.0,3.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.052941,0.158824,-0.0,538,195,14,291.4,92.8,4.6,224.550217,78.023714,5.458938,395.0,96.0,3.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0

In [469]:
to_merge = to_merge.groupby(['merchant_id', 'time_period']).sum().unstack().fillna(0).stack().reset_index()

In [471]:
to_merge = to_merge.pivot_table(values=['clicks_merchant', 'purchases_merchant', 'favourites_merchant'], index='merchant_id', columns='time_period')

In [472]:
to_merge.columns = ['_period_'.join(str(x) for x in col) for col in to_merge.columns.values]

In [473]:
df = df.merge(to_merge, on='merchant_id', how='left')

In [474]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline,clicks_merchant_period_max,purchases_merchant_period_max,favourites_merchant_period_max,clicks_merchant_period_mean,purchases_merchant_period_mean,favourites_merchant_period_mean,clicks_merchant_period_std,purchases_merchant_period_std,favourites_merchant_period_std,clicks_merchant_period_median,purchases_merchant_period_median,favourites_merchant_period_median,clicks_merchant_period_0,clicks_merchant_period_1,clicks_merchant_period_2,clicks_merchant_period_3,clicks_merchant_period_4,clicks_merchant_period_5,favourites_merchant_period_0,favourites_merchant_period_1,favourites_merchant_period_2,favourites_merchant_period_3,favourites_merchant_period_4,favourites_merchant_period_5,purchases_merchant_period_0,purchases_merchant_period_1,purchases_merchant_period_2,purchases_merchant_period_3,purchases_merchant_period_4,purchases_merchant_period_5
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0,154,7,5,79.0,3.5,2.5,106.066017,4.949747,3.535534,79.0,3.5,2.5,0.0,0.0,0.0,0.0,4.0,154.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.017647,-0.0,80,2,6,42.4,0.6,2.0,27.272697,0.894427,2.449490,43.0,0.0,2.0,0.0,5.0,51.0,80.0,43.0,33.0,0.0,0.0,2.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0

###### merchants trendline

In [475]:
purchase_col = [col for col in df if col.startswith('purchases_merchant_period') and col[-1].isdigit()]

In [476]:
x = np.array([int(col[-1]) for col in purchase_col]).reshape(1, -1)

In [477]:
y = df[purchase_col].to_numpy()

In [478]:
df['purchase_merchant_trendline'] = trendline(x, y)

In [479]:
clicks_col = [col for col in df if col.startswith('clicks_merchant_period') and col[-1].isdigit()]

In [480]:
y = df[clicks_col].to_numpy()

In [481]:
df['clicks_merchant_trendline'] = trendline(x, y)

In [482]:
favourites_col = [col for col in df if col.startswith('favourites_merchant_period') and col[-1].isdigit()]

In [483]:
y = df[favourites_col].to_numpy()

In [484]:
df['favourites_merchant_trendline'] = trendline(x, y)

In [485]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline,clicks_merchant_period_max,purchases_merchant_period_max,favourites_merchant_period_max,clicks_merchant_period_mean,purchases_merchant_period_mean,favourites_merchant_period_mean,clicks_merchant_period_std,purchases_merchant_period_std,favourites_merchant_period_std,clicks_merchant_period_median,purchases_merchant_period_median,favourites_merchant_period_median,clicks_merchant_period_0,clicks_merchant_period_1,clicks_merchant_period_2,clicks_merchant_period_3,clicks_merchant_period_4,clicks_merchant_period_5,favourites_merchant_period_0,favourites_merchant_period_1,favourites_merchant_period_2,favourites_merchant_period_3,favourites_merchant_period_4,favourites_merchant_period_5,purchases_merchant_period_0,purchases_merchant_period_1,purchases_merchant_period_2,purchases_merchant_period_3,purchases_merchant_period_4,purchases_merchant_period_5,purchase_merchant_trendline,clicks_merchant_trendline,favourites_merchant_trendline
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0,154,7,5,79.0,3.5,2.5,106.066017,4.949747,3.535534,79.0,3.5,2.5,0.0,0.0,0.0,0.0,4.0,154.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0,-0.617647,-13.800000,-0.441176
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.942222,0.013333,0.044444,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.017647,-0.0,80,2,6,42.4,0.6,2.0,27.272697,0.894427,2.449490,43.0,0.0,2.0,0.0,5.0,51.0,80.0,43.0,33.0,0.0,0.0,2.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,-0.229412,-5.435294,-0.176471
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.2500

###### user merchant

In [486]:
users_merchants_time = df_user_log.groupby(['user_id', 'merchant_id', 'time_period'])

In [494]:
to_merge = users_merchants_time['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_user_merchant',
    1: 'carts_user_merchant',
    2: 'purchases_user_merchant',
    3: 'favourites_user_merchant'
    }).reset_index()

In [496]:
to_merge = to_merge.groupby(['user_id', 'merchant_id', 'time_period']).sum().unstack().fillna(0).stack().reset_index()

In [497]:
to_merge

action_type,user_id,merchant_id,time_period,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant
0,170,4255,0,0.0,0.0,0.0
1,170,4255,1,0.0,0.0,0.0
2,170,4255,2,1.0,0.0,0.0
3,170,4255,3,1.0,0.0,0.0
4,170,4255,4,0.0,0.0,0.0
...,...,...,...,...,...,...
14845,424156,4255,1,0.0,0.0,0.0
14846,424156,4255,2,0.0,0.0,0.0
14847,424156,4255,3,3.0,0.0,0.0
14848,424156,4255,4,0.0,0.0,0.0


In [498]:
to_merge = to_merge.pivot_table(values=['clicks_user_merchant', 'purchases_user_merchant', 'favourites_user_merchant'], index=['user_id', 'merchant_id'], columns='time_period')

In [499]:
to_merge

action_type         clicks_user_merchant                           \
time_period                            0    1    2    3    4    5   
user_id merchant_id                                                 
170     4255                         0.0  0.0  1.0  1.0  0.0  0.0   
393     4344                         0.0  0.0  0.0  0.0  0.0  1.0   
699     3491                         1.0  0.0  0.0  0.0  0.0  0.0   
784     983                          0.0  0.0  1.0  0.0  0.0  0.0   
867     4255                         0.0  0.0  0.0  0.0  0.0  1.0   
...                                  ...  ...  ...  ...  ...  ...   
422954  3462                         0.0  0.0  0.0  0.0  1.0  0.0   
422967  1461                         2.0  0.0  0.0  0.0  0.0  0.0   
423225  1078                         0.0  0.0  0.0  0.0  0.0  1.0   
423387  983                          0.0  1.0  0.0  0.0  0.0  0.0   
424156  4255                         0.0  0.0  0.0  3.0  0.0  0.0   

action_type         favourites_user_merchant                           \
time_period                                0    1    2    3    4    5   
user_id merchant_id                                                     
170     4255                             0.0  0.0  0.0  0.0  0.0  0.0   
393     4344                             0.0  0.0  0.0  0.0  0.0  0.0   
699     3491                             0.0  0.0  0.0  0.0  0.0  0.0   
784     983                              0.0  0.0  0.0  0.0  0.0  0.0   
867     4255                             0.0  0.0  0.0  0.0  0.0  0.0   
...                                      ...  ...  ...  ...  ...  ...   
422954  3462                             0.0  0.0  0.0  0.0  0.0  0.0   
422967  1461                             0.0  0.0  0.0  0.0  0.0  0.0   
423225  1078                             0.0  0.0  0.0  0.0  0.0  0.0   
423387  983                              0.0  0.0  0.0  0.0  0.0  0.0   
424156  4255                             0.0  0.0  0.0  0.0  0.0  0.0   

action_type         purchases_user_merchant                           
time_period                               0    1    2    3    4    5  
user_id merchant_id                                                   
170     4255                            0.0  0.0  0.0  0.0  0.0  0.0  
393     4344                            0.0  0.0  0.0  0.0  0.0  0.0  
699     3491                            0.0  0.0  0.0  0.0  0.0  0.0  
784     983                             0.0  0.0  0.0  0.0  0.0  0.0  
867     4255                            0.0  0.0  0.0  0.0  0.0  0.0  
...                                     ...  ...  ...  ...  ...  ...  
422954  3462                            0.0  0.0  0.0  0.0  0.0  0.0  
422967  1461                            0.0  0.0  0.0  0.0  0.0  0.0  
423225  1078                            0.0  0.0  0.0  0.0  0.0  0.0  
423387  983                             0.0  0.0  0.0  0.0  0.0  0.0  
424156  4255                            0.0  0.0  0.0  0.0  0.0  0.0  

[2475 rows x 18 columns]

In [500]:
to_merge.columns = ['_period_'.join(str(x) for x in col) for col in to_merge.columns.values]

In [501]:
to_merge

,,clicks_user_merchant_period_0,clicks_user_merchant_period_1,clicks_user_merchant_period_2,clicks_user_merchant_period_3,clicks_user_merchant_period_4,clicks_user_merchant_period_5,favourites_user_merchant_period_0,favourites_user_merchant_period_1,favourites_user_merchant_period_2,favourites_user_merchant_period_3,favourites_user_merchant_period_4,favourites_user_merchant_period_5,purchases_user_merchant_period_0,purchases_user_merchant_period_1,purchases_user_merchant_period_2,purchases_user_merchant_period_3,purchases_user_merchant_period_4,purchases_user_merchant_period_5
user_id,merchant_id,,,,,,,,,,,,,,,,,,
170,4255,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
393,4344,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
699,3491,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
784,983,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
867,4255,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422954,3462,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
422967,1461,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
423225,1078,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [502]:
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')

In [503]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline,clicks_merchant_period_max,purchases_merchant_period_max,favourites_merchant_period_max,clicks_merchant_period_mean,purchases_merchant_period_mean,favourites_merchant_period_mean,clicks_merchant_period_std,purchases_merchant_period_std,favourites_merchant_period_std,clicks_merchant_period_median,purchases_merchant_period_median,favourites_merchant_period_median,clicks_merchant_period_0,clicks_merchant_period_1,clicks_merchant_period_2,clicks_merchant_period_3,clicks_merchant_period_4,clicks_merchant_period_5,favourites_merchant_period_0,favourites_merchant_period_1,favourites_merchant_period_2,favourites_merchant_period_3,favourites_merchant_period_4,favourites_merchant_period_5,purchases_merchant_period_0,purchases_merchant_period_1,purchases_merchant_period_2,purchases_merchant_period_3,purchases_merchant_period_4,purchases_merchant_period_5,purchase_merchant_trendline,clicks_merchant_trendline,favourites_merchant_trendline,clicks_user_merchant_period_0,clicks_user_merchant_period_1,clicks_user_merchant_period_2,clicks_user_merchant_period_3,clicks_user_merchant_period_4,clicks_user_merchant_period_5,favourites_user_merchant_period_0,favourites_user_merchant_period_1,favourites_user_merchant_period_2,favourites_user_merchant_period_3,favourites_user_merchant_period_4,favourites_user_merchant_period_5,purchases_user_merchant_period_0,purchases_user_merchant_period_1,purchases_user_merchant_period_2,purchases_user_merchant_period_3,purchases_user_merchant_period_4,purchases_user_merchant_period_5
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0,154,7,5,79.0,3.5,2.5,106.066017,4.949747,3.535534,79.0,3.5,2.5,0.0,0.0,0.0,0.0,4.0,154.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0,-0.617647,-13.800000,-0.441176,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,252288,3990,-1,0,0,0,1,0,0,0,0

###### user-merchants trendline

In [504]:
purchase_col = [col for col in df if col.startswith('purchases_user_merchant_period') and col[-1].isdigit()]

In [505]:
x = np.array([int(col[-1]) for col in purchase_col]).reshape(1, -1)

In [506]:
y = df[purchase_col].to_numpy()

In [507]:
df['purchase_user_merchant_trendline'] = trendline(x, y)

In [508]:
clicks_col = [col for col in df if col.startswith('clicks_user_merchant_period') and col[-1].isdigit()]

In [509]:
y = df[clicks_col].to_numpy()

In [510]:
df['clicks_user_merchant_trendline'] = trendline(x, y)

In [514]:
favourites_col = [col for col in df if col.startswith('favourites_user_merchant_period') and col[-1].isdigit()]

In [515]:
y = df[favourites_col].to_numpy()

In [516]:
df['favourites_user_merchant_trendline'] = trendline(x, y)

In [517]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5,purchase_user_trendline,clicks_user_trendline,favourites_user_trendline,clicks_merchant_period_max,purchases_merchant_period_max,favourites_merchant_period_max,clicks_merchant_period_mean,purchases_merchant_period_mean,favourites_merchant_period_mean,clicks_merchant_period_std,purchases_merchant_period_std,favourites_merchant_period_std,clicks_merchant_period_median,purchases_merchant_period_median,favourites_merchant_period_median,clicks_merchant_period_0,clicks_merchant_period_1,clicks_merchant_period_2,clicks_merchant_period_3,clicks_merchant_period_4,clicks_merchant_period_5,favourites_merchant_period_0,favourites_merchant_period_1,favourites_merchant_period_2,favourites_merchant_period_3,favourites_merchant_period_4,favourites_merchant_period_5,purchases_merchant_period_0,purchases_merchant_period_1,purchases_merchant_period_2,purchases_merchant_period_3,purchases_merchant_period_4,purchases_merchant_period_5,purchase_merchant_trendline,clicks_merchant_trendline,favourites_merchant_trendline,clicks_user_merchant_period_0,clicks_user_merchant_period_1,clicks_user_merchant_period_2,clicks_user_merchant_period_3,clicks_user_merchant_period_4,clicks_user_merchant_period_5,favourites_user_merchant_period_0,favourites_user_merchant_period_1,favourites_user_merchant_period_2,favourites_user_merchant_period_3,favourites_user_merchant_period_4,favourites_user_merchant_period_5,purchases_user_merchant_period_0,purchases_user_merchant_period_1,purchases_user_merchant_period_2,purchases_user_merchant_period_3,purchases_user_merchant_period_4,purchases_user_merchant_period_5,purchase_user_merchant_trendline,clicks_user_merchant_trendline,favourites_user_merchant_trendline
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000000,-0.088235,-0.0,154,7,5,79.0,3.5,2.5,106.066017,4.949747,3.535534,79.0,3.5,2.5,0.0,0.0,0.0,0.0,4.0,154.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0,-0.617647,-13.800000,-0.441176,0.0,0

In [518]:
df.to_csv('features_version_1.csv',index=False)

In [459]:
len(df)

4644

In [422]:
df[df['user_id']==210048]

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max,clicks_user_period_mean,purchases_user_period_mean,favourites_user_period_mean,clicks_user_period_std,purchases_user_period_std,favourites_user_period_std,clicks_user_period_median,purchases_user_period_median,favourites_user_period_median,clicks_user_period_0,clicks_user_period_1,clicks_user_period_2,clicks_user_period_3,clicks_user_period_4,clicks_user_period_5,favourites_user_period_0,favourites_user_period_1,favourites_user_period_2,favourites_user_period_3,favourites_user_period_4,favourites_user_period_5,purchases_user_period_0,purchases_user_period_1,purchases_user_period_2,purchases_user_period_3,purchases_user_period_4,purchases_user_period_5
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.75,0.25,0.0,0.75,0.25,0.0,0.749486,0.238683,0.011831,0,3,1,0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [421]:
# duff[(duff['merchant_id']==4255) & (duff['user_id']==867)]
# duff[(duff['merchant_id']==4255)]
duff[(duff['action_type']==2)]

,user_id,age_range,gender,merchant_id,label,item_id,category_id,brand_id,time_stamp,action_type
2,210048,3,1,4255,-1,866,1213,1573,711,2
17,279168,2,1,1110,-1,857,1112,2187,813,2
22,285312,0,0,3058,-1,809,308,8473,1030,2
33,68994,4,1,4255,-1,866,1213,1573,813,2
41,105603,7,1,4255,-1,866,1213,1573,814,2
...,...,...,...,...,...,...,...,...,...,...
4555,415088,3,1,4255,-1,866,1213,1573,712,2
4569,344177,5,1,4255,-1,866,1213,1573,813,2
4577,32882,6,1,4255,-1,866,1213,1573,710,2
4639,230015,4,0,4255,-1,866,1213,1573,710,2


In [452]:
df_user_log[df_user_log['merchant_id']==4945]

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type,time_period
1284,98391,820,11,4945,7069,143,0,4
1679,184474,820,11,4945,7069,140,0,4
2734,253519,820,11,4945,7069,110,0,3
2815,135007,820,11,4945,7069,58,0,1
2816,135007,820,11,4945,7069,58,0,1
2837,330598,820,11,4945,7069,51,0,1
3286,20906,820,11,4945,7069,57,0,1
3846,78848,820,11,4945,7069,94,0,3
3880,39173,820,11,4945,7069,160,0,5
3968,155927,820,11,4945,7069,45,0,1


In [360]:
df

,user_id,merchant_id,label,gender,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,items_user,categories_user,merchants_user,brands_user,dates_user,action_types_user,periods_user,users_merchant,items_merchant,categories_merchant,brands_merchant,dates_merchant,action_types_merchant,periods_merchant,items_user_merchant,categories_user_merchant,brands_user_merchant,dates_user_merchant,action_types_user_merchant,periods_user_merchant,clicks_user,purchases_user,favourites_user,clicks_merchant,purchases_merchant,favourites_merchant,clicks_user_merchant,purchases_user_merchant,favourites_user_merchant,clicks_in_merchant_ratio_perspective,purchases_in_merchant_ratio_perspective,favourites_in_merchant_ratio_perspective,clicks_by_user_ratio_perspective,purchases_by_user_ratio_perspective,favourites_by_user_ratio_perspective,clicks_user_ratio,purchases_user_ratio,favourites_user_ratio,clicks_user_merchant_ratio,purchases_user_merchant_ratio,favourites_user_merchant_ratio,clicks_merchant_ratio,purchases_merchant_ratio,favorites_merchant_ratio,interval,clicks_user_period_max,purchases_user_period_max,favourites_user_period_max
0,34944,2116,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,115,1,1,1,24,3,2,1,1,1,1,1,1,1,0,0,158,7,5,1,0,0,1.0,0.0,0.0,0.006329,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.929412,0.041176,0.029412,0,1,0,0
1,252288,3990,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,155,1,1,1,80,3,5,1,1,1,1,1,1,1,0,0,212,3,10,1,0,0,1.0,0.0,0.0,0.004717,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.942222,0.013333,0.044444,0,1,0,0
2,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0
3,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0
4,210048,4255,-1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,3,1,0,1457,464,23,3,1,0,1.0,1.0,0.0,0.002059,0.002155,0.0,0.750000,0.250000,0.0,0.750000,0.250000,0.0,0.749486,0.238683,0.011831,0,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639,230015,4255,-1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,2,1,0,1457,464,23,2,1,0,1.0,1.0,0.0,0.001373,0.002155,0.0,0.666667,0.333333,0.0,0.666667,0.333333,0.0,0.749486,0.238683,0.011831,0,2,1,0
4640,230015,4255,-1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,2,1,681,1,1,1,103,3,5,1,1,1,1,2,1,2,1,0,1457,464,23,2,1,0,1.0,1.0,0.0,0.001373,0.002155,0.0,0.666667,0.333333,0.0,0.666667,0.333333,0.0,0.749486,0.238683,0.011831,0,2,1,0
4641,64895,3491,-1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,22,1,1,1,11,1,1,1,1,1,1,1,1,1,0,0,23,0,0,1,0,0,1.0,0.0,0.0,0.043478,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0,1,0,0
4642,5759,4255,-1,0,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1,681,1,1,1,103,3,5,1,1,1,1,1,1,0,1,0,1457,464,23,0,1,0,0.0,1.0,0.0,0.000000,0.002155,0.0,0.000000,1.000000,0.0,0.000000,1.000000,0.0,0.749486,0.238683,0.011831,0,0,1,0
